### Analyzing insider trad

In [254]:
#import libraries
import numpy as np
import pandas as pd
import requests
import json
import datetime
from tqdm import tqdm
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import math

### Collecting insider trades data and stock price data for Microsoft from https://site.financialmodelingprep.com/

In [255]:
 def get_insider_trades(start_date, symbol, apikey):
        
        start_date_dt=datetime.datetime.strptime(start_date, '%Y-%m-%d')
        collected_month=[]
        
        page=0
        all_info_trades=pd.DataFrame()
        while True:
            endpoint=f'https://financialmodelingprep.com/api/v4/insider-trading?symbol={symbol}&page={page}&apikey={apikey}'
            response=requests.get(endpoint)
            
            temp_it=response.json()
            
            temp_itdf=pd.DataFrame.from_dict(temp_it)
           
            if len(temp_itdf)>0:
                
                
                temp_itdf['filingDate']=pd.to_datetime(temp_itdf['filingDate'])
                temp_itdf.set_index('filingDate',inplace=True)
                date_collected=list(temp_itdf.groupby([temp_itdf.index.year,temp_itdf.index.month])['acquistionOrDisposition'].count().sort_index(ascending=False).index)
                for date in date_collected:
                    if start_date_dt<=datetime.datetime(*(date),1):
                        year_month=f'{str(date[0])}-{str(date[1])}'
                        if year_month not in collected_month:
                            print(f'Data for {year_month} collected')
                        collected_month.append(year_month)
                
                all_info_trades=pd.concat([all_info_trades, temp_itdf])
                if all_info_trades.index[-1]<=start_date_dt:
                    all_info_trades=all_info_trades.loc[all_info_trades.index>=start_date_dt]
                    break
                
            else:
                if len(all_info_trades)>0:
                    print(f"All accessible data for insider trades is collected. Last record is from {all_info_trades.index[-1]}")
                else:
                    print(f'No insader trades data for symbol {symbol}')
                    return 0
                break
            
            
                
            
            page+=1
        
        
      
        all_info_trades=all_info_trades.replace(r'^\s*$', np.nan, regex=True)
        all_info_trades=all_info_trades.replace(r'^\s*$', np.nan, regex=True)
        all_info_trades['transactionDate']=pd.to_datetime(all_info_trades['transactionDate'])
        return all_info_trades
        

In [256]:
#your api key
apikey='<YOUR FMP API KEY>'
start_date='2019-01-01'

In [257]:
#collecting insider trades data for Microsoft
trades_data=get_insider_trades(start_date,'MSFT',apikey)

Data for 2024-2 collected
Data for 2024-1 collected
Data for 2023-12 collected
Data for 2023-11 collected
Data for 2023-10 collected
Data for 2023-9 collected
Data for 2023-8 collected
Data for 2023-7 collected
Data for 2023-6 collected
Data for 2023-5 collected
Data for 2023-4 collected
Data for 2023-3 collected
Data for 2023-2 collected
Data for 2023-1 collected
Data for 2022-12 collected
Data for 2022-11 collected
Data for 2022-10 collected
Data for 2022-9 collected
Data for 2022-7 collected
Data for 2022-6 collected
Data for 2022-4 collected
Data for 2022-3 collected
Data for 2022-2 collected
Data for 2022-1 collected
Data for 2021-12 collected
Data for 2021-11 collected
Data for 2021-10 collected
Data for 2021-9 collected
Data for 2021-7 collected
Data for 2021-6 collected
Data for 2021-5 collected
Data for 2021-4 collected
Data for 2021-3 collected
Data for 2021-2 collected
Data for 2021-1 collected
Data for 2020-12 collected
Data for 2020-11 collected
Data for 2020-10 collected


In [258]:
trades_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 858 entries, 2024-02-06 18:14:02 to 2019-01-28 18:09:11
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   symbol                   858 non-null    object        
 1   transactionDate          858 non-null    datetime64[ns]
 2   reportingCik             858 non-null    object        
 3   transactionType          847 non-null    object        
 4   securitiesOwned          858 non-null    float64       
 5   companyCik               858 non-null    object        
 6   reportingName            858 non-null    object        
 7   typeOfOwner              858 non-null    object        
 8   acquistionOrDisposition  858 non-null    object        
 9   formType                 858 non-null    object        
 10  securitiesTransacted     858 non-null    float64       
 11  price                    858 non-null    float64       
 12 

In [259]:
trades_data

,symbol,transactionDate,reportingCik,transactionType,securitiesOwned,companyCik,reportingName,typeOfOwner,acquistionOrDisposition,formType,securitiesTransacted,price,securityName,link
filingDate,,,,,,,,,,,,,,
2024-02-06 18:14:02,MSFT,2024-02-02,0001193119,S-Sale,574129.2701,0000789019,SMITH BRADFORD L,officer: Vice Chair and President,D,4,45000.0,411.7784,Common Stock,https://www.sec.gov/Archives/edgar/data/789019...
2024-02-06 18:14:02,MSFT,2024-02-05,0001193119,S-Sale,573051.2701,0000789019,SMITH BRADFORD L,officer: Vice Chair and President,D,4,1078.0,404.8921,Common Stock,https://www.sec.gov/Archives/edgar/data/789019...
2024-02-06 18:14:02,MSFT,2024-02-05,0001193119,S-Sale,571626.2701,0000789019,SMITH BRADFORD L,officer: Vice Chair and President,D,4,1425.0,405.7466,Common Stock,https://www.sec.gov/Archives/edgar/data/789019...
2024-02-06 18:14:02,MSFT,2024-02-05,0001193119,S-Sale,571326.2701,0000789019,SMITH BRADFORD L,officer: Vice Chair and President,D,4,300.0,406.8433,Common Stock,https://www.sec.gov/Archives/edgar/data/789019...
2024-02-06 18:14:02,MSFT,2024-02-05,0001193119,S-Sale,571126.2701,0000789019,SMITH BRADFORD L,officer: Vice Chair and President,D,4,200.0,408.1000,Common Stock,https://www.sec.gov/Archives/edgar/data/789019...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-01 18:09:52,MSFT,2019-01-31,0001039894,A-Award,96099.0000,0000789019,NOSKI CHARLES H,director,A,4,478.0,0.0000,Restricted Stock Units,https://www.sec.gov/Archives/edgar/data/789019...
2019-02-01 18:07:46,MSFT,2019-01-31,0001529370,A-Award,16402.0000,0000789019,List-Stoll Teri,director,A,4,813.0,0.0000,Restricted Stock Units,https://www.sec.gov/Archives/edgar/data/789019...
2019-02-01 18:06:00,MSFT,2019-01-31,0001377489,A-Award,3134.0000,0000789019,Johnston Hugh F,director,A,4,478.0,0.0000,Common Stock,https://www.sec.gov/Archives/edgar/data/789019...


In [260]:
#collecting price data
def get_stock_prices(start_date, symbol, apikey, end_date=None):
        
    if end_date==None:
        end_date=datetime.datetime.now()
        end_date=end_date.strftime('%Y-%m-%d')

    endpoint=f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&to={end_date}&apikey={apikey}'
    response=requests.get(endpoint)

    df=pd.DataFrame.from_dict(response.json()['historical'])
    df['date']=pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

In [261]:
price_data=get_stock_prices(start_date,'MSFT',apikey)

In [262]:
price_data

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
date,,,,,,,,,,,,
2024-02-22,410.19,412.83,408.57,411.65,411.65,26338613,25452946,1.46,0.35593,410.78,"February 22, 24",0.003559
2024-02-21,400.17,402.29,397.22,402.18,402.18,18576472,18631100,2.01,0.50229,400.29,"February 21, 24",0.005023
2024-02-20,403.24,404.49,398.01,402.79,402.79,24307915,24307900,-0.45,-0.11160,401.44,"February 20, 24",-0.001116
2024-02-16,407.96,408.29,403.44,404.06,404.06,22296495,22281100,-3.90,-0.95598,405.20,"February 16, 24",-0.009560
2024-02-15,408.14,409.13,404.29,406.56,406.56,21825525,21825500,-1.58,-0.38712,406.09,"February 15, 24",-0.003871
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-08,103.04,103.97,101.71,102.80,97.44,31514400,31514400,-0.24,-0.23292,102.83,"January 08, 19",-0.002329
2019-01-07,101.64,103.27,100.98,102.06,96.74,35656100,35656100,0.42,0.41322,102.10,"January 07, 19",0.004132
2019-01-04,99.72,102.51,98.93,101.93,96.61,44060600,44060600,2.21,2.22000,101.12,"January 04, 19",0.022200


### How insider trades impact stock prices only in point of view Acquistion or Disposition.

In [263]:
#merge and grouping data with defined interval

def merge_price_aord(price_data,trades_data,resample):
    
    
    
    df_aord_pivot=pd.DataFrame()
    price_df=price_data.copy()
    trades_df=trades_data.copy()
    company=trades_df['symbol'].iloc[1]
    
    trades_df.reset_index(inplace=True)
    trades_df['filingDate']=pd.DatetimeIndex(trades_df['filingDate']).date
    df_a_or_d=trades_df.groupby(['filingDate','acquistionOrDisposition'])['securitiesTransacted'].sum()
    df_a_or_d=df_a_or_d.reset_index()
    df_aord_pivot=df_a_or_d.pivot_table(index=['filingDate'], columns=['acquistionOrDisposition'],
                                       values='securitiesTransacted').fillna(0)

    df_aord_pivot.index=pd.to_datetime(df_aord_pivot.index)


    if resample!='D':
        df_aord_pivot=df_aord_pivot.resample(resample).agg('sum')
        price_df=price_df.resample(resample).agg({'open':'first','high':'max','low':'min','close':'last', 'volume':'sum'})

    price_ad_df=pd.concat([price_df[['open','high','low','close','volume']],df_aord_pivot], axis=1).fillna(0)
    price_ad_df.attrs['symbol'] = company
    price_ad_df.attrs['freq'] = resample
    
    
    
    return price_ad_df

In [264]:
#function to generate plot
def plot_trades_on_price_chart(data, log_scale=False):
        
        
        company=data.attrs['symbol']
        resample=data.attrs['freq']
        title=f"Type of insider trades |{resample}| related to {company} price, from {data.index[0]} to {data.index[-1]}"
        fig_price=make_subplots(rows=3,cols=1, row_heights=[0.6,0.2,0.2],
                        vertical_spacing=0.04, subplot_titles=(f'{company} price','Acquistion','Disposition'))
        
        
        fig_price.update_layout(height=1000)
        fig_price.add_trace(go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'],
                name='price'), row=1, col=1)
        
        fig_price.add_trace(go.Bar(x=data.index, y=data['A'], name='Acquistion'), row=2,col=1)
                
        fig_price.add_trace(go.Bar(x=data.index, y=data['D'], name='Disposition'), row=3,col=1)
        
        fig_price.update_layout(title=title, xaxis_rangeslider_visible=False,newshape_line_color='black')
        if resample=='D':
            
            fig_price.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])
            
        if log_scale==True:
            fig_price.update_yaxes(type='log', row=2,col=1)
            fig_price.update_yaxes(type='log',row=3,col=1)
            fig_price.layout.annotations[1].update(text='Acquistion log scale')
            fig_price.layout.annotations[2].update(text='Disposition log scale')

        fig_price.update_xaxes(spikemode='across+marker')
        fig_price.update_traces(xaxis="x1")
        
        fig_price.show(config={'modeBarButtonsToAdd':['drawline',
                                            'drawopenpath',
                                            'drawclosedpath',
                                            'drawcircle',
                                            'drawrect',
                                            'eraseshape'
                                       ]})
        
        

In [265]:
#merge data with interval of one week
data_ad=merge_price_aord(price_data,trades_data,resample='W')


In [266]:
data_ad

,open,high,low,close,volume,A,D,I
2019-01-06,99.550,102.51,97.20,101.93,121969000,0.000,0.0,0.0
2019-01-13,101.640,104.88,100.98,102.80,157833100,0.000,0.0,0.0
2019-01-20,101.900,107.90,101.26,107.71,155699200,0.000,0.0,0.0
2019-01-27,106.750,107.88,104.86,107.17,112636000,0.000,0.0,0.0
2019-02-03,106.260,106.48,102.17,102.78,201611200,7958.000,4637.0,0.0
...,...,...,...,...,...,...,...,...
2024-01-28,400.020,407.01,393.59,403.93,111234300,0.000,0.0,0.0
2024-02-04,406.060,415.32,397.21,411.22,164761600,2106.882,44.4,0.0
2024-02-11,409.900,420.82,402.91,420.55,109311001,0.000,48303.0,0.0
2024-02-18,420.555,420.74,403.39,404.06,113544267,0.000,0.0,0.0


In [267]:
plot_trades_on_price_chart(data_ad)

### More detailed classifiication of insider trades

In [268]:
endpoint=f'https://financialmodelingprep.com/api/v4/insider-trading-transaction-type?apikey={apikey}'

In [269]:
response=requests.get(endpoint)

In [270]:
ttypes=response.json()

In [271]:
ttypes

['A-Award',
 'C-Conversion',
 'D-Return',
 'E-ExpireShort',
 'F-InKind',
 'G-Gift',
 'H-ExpireLong',
 'I-Discretionary',
 'J-Other',
 'L-Small',
 'M-Exempt',
 'O-OutOfTheMoney',
 'P-Purchase',
 'S-Sale',
 'U-Tender',
 'W-Will',
 'X-InTheMoney',
 'Z-Trust']

In [272]:
def merge_data_ttypes(trades_data,price_data,resample):
    
   
    df_aord_pivot=pd.DataFrame()
    price_df=price_data.copy()
    trades_df=trades_data.copy()
    company=trades_df['symbol'].iloc[1]
    
    trades_df.reset_index(inplace=True)
    trades_df['filingDate']=pd.DatetimeIndex(trades_df['filingDate']).date
    df_a_or_d=trades_df.groupby(['filingDate','acquistionOrDisposition', 'transactionType'])['securitiesTransacted'].sum()
    df_a_or_d=df_a_or_d.reset_index()
    df_aord_pivot=df_a_or_d.pivot_table(index=['filingDate'], columns=['acquistionOrDisposition','transactionType'],
                                       values='securitiesTransacted').fillna(0)

    df_aord_pivot.index=pd.to_datetime(df_aord_pivot.index)


    if resample!='D':
        df_aord_pivot=df_aord_pivot.resample(resample).agg('sum')
        price_df=price_df.resample(resample).agg({'open':'first','high':'max','low':'min','close':'last', 'volume':'sum'})

    price_ttypes_df=pd.concat([price_df[['open','high','low','close','volume']],df_aord_pivot], axis=1).fillna(0)
    price_ttypes_df.attrs['symbol'] = company
    price_ttypes_df.attrs['freq'] = resample
    
    return price_ttypes_df
    

In [273]:
df_ttypes=merge_data_ttypes(trades_data,price_data, 'W')

In [274]:
df_ttypes

,open,high,low,close,volume,"(A, A-Award)","(A, G-Gift)","(A, J-Other)","(A, P-Purchase)","(D, F-InKind)","(D, G-Gift)","(D, J-Other)","(D, S-Sale)"
2019-01-06,99.550,102.51,97.20,101.93,121969000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-13,101.640,104.88,100.98,102.80,157833100,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-20,101.900,107.90,101.26,107.71,155699200,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-27,106.750,107.88,104.86,107.17,112636000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-02-03,106.260,106.48,102.17,102.78,201611200,7958.000,0.0,0.0,0.0,4637.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-28,400.020,407.01,393.59,403.93,111234300,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-04,406.060,415.32,397.21,411.22,164761600,2106.882,0.0,0.0,0.0,44.4,0.0,0.0,0.0
2024-02-11,409.900,420.82,402.91,420.55,109311001,0.000,0.0,0.0,0.0,0.0,0.0,0.0,48303.0
2024-02-18,420.555,420.74,403.39,404.06,113544267,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [275]:
def plot_ttypes_on_price_chart(data, log_scale=False):
    
    
    company=data.attrs['symbol']
    resample=data.attrs['freq']
    title=f"Type of insider trades |{resample}| related to {company} price, from {data.index[0]} to {data.index[-1]}"
    fig_price=make_subplots(rows=3,cols=1, row_heights=[0.6,0.2,0.2],
                        vertical_spacing=0.04, subplot_titles=(f'{company} price', 'Acquistion','Disposition'))
        
        
    fig_price.update_layout(height=1000)
    fig_price.add_trace(go.Candlestick(x=data.index,
            open=data['open'],
            high=data['high'],
            low=data['low'],
            close=data['close'],
            name='price'), row=1, col=1)



    for col in data.columns:
        if type(col)!=str:
            if col[0]=='A':
                fig_price.add_trace(go.Bar(x=data.index, y=data[col], name=f'{col[1]}-{col[0]}'), row=2,col=1)
            if col[0]=='D':
                fig_price.add_trace(go.Bar(x=data.index, y=data[col], name=f'{col[1]}-{col[0]}'), row=3,col=1)

    fig_price.update_layout(title=title, xaxis_rangeslider_visible=False, barmode='stack',
    newshape_line_color='black')
    if resample=='D':

        fig_price.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])
    
    fig_price.update_xaxes(spikemode='across+marker')
    if log_scale==True:
        fig_price.update_yaxes(type='log', row=2,col=1)
        fig_price.update_yaxes(type='log',row=3,col=1)
        fig_price.layout.annotations[1].update(text='Acquistion log scale')
        fig_price.layout.annotations[2].update(text='Disposition log scale')


    fig_price.update_traces(xaxis="x1")
    fig_price.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                   ]})
        

In [276]:
plot_ttypes_on_price_chart(df_ttypes)

### Classification of insider trades in respect of who made transaction

In [277]:
def merge_data_towner(trades_data,price_data,resample):
    
     
    df_aord_pivot=pd.DataFrame()
    price_df=price_data.copy()
    trades_df=trades_data.copy()
    company=trades_df['symbol'].iloc[1]

    
    trades_df.reset_index(inplace=True)
    trades_df['filingDate']=pd.DatetimeIndex(trades_df['filingDate']).date
    df_a_or_d=trades_df.groupby(['filingDate','acquistionOrDisposition', 'typeOfOwner'])['securitiesTransacted'].sum()
    df_a_or_d=df_a_or_d.reset_index()
    df_aord_pivot=df_a_or_d.pivot_table(index=['filingDate'], columns=['acquistionOrDisposition','typeOfOwner'],
                                       values='securitiesTransacted').fillna(0)

    df_aord_pivot.index=pd.to_datetime(df_aord_pivot.index)


    if resample!='D':
        df_aord_pivot=df_aord_pivot.resample(resample).agg('sum')
        price_df=price_df.resample(resample).agg({'open':'first','high':'max','low':'min','close':'last', 'volume':'sum'})

    price_towner_df=pd.concat([price_df[['open','high','low','close','volume']],df_aord_pivot], axis=1).fillna(0)
    price_towner_df.attrs['symbol'] = company
    price_towner_df.attrs['freq'] = resample
    
    return price_towner_df

In [278]:
df_towner=merge_data_towner(trades_data,price_data,resample='W')

In [279]:
df_towner.columns

Index([                                             'open',
                                                    'high',
                                                     'low',
                                                   'close',
                                                  'volume',
                                         ('A', 'director'),
       ('A', 'director, officer: Chief Executive Officer'),
                                       ('A', 'director: '),
                ('A', 'officer: Chief Accounting Officer'),
                 ('A', 'officer: Chief Executive Officer'),
           ('A', 'officer: EVP & Chief Financial Officer'),
                            ('A', 'officer: EVP, Bus Dev'),
               ('A', 'officer: EVP, Business Development'),
           ('A', 'officer: EVP, Chief Commercial Officer'),
            ('A', 'officer: EVP, Chief Financial Officer'),
            ('A', 'officer: EVP, Chief Marketing Officer'),
                    ('A', 'officer: EVP,

In [280]:
def plot_towner_on_price_chart(data, log_scale=False):
    
    
    company=data.attrs['symbol']
    resample=data.attrs['freq']
    title=f"Type of insider trades |{resample}| related to {company} price, from {data.index[0]} to {data.index[-1]}"
    fig_price=make_subplots(rows=3,cols=1, row_heights=[0.6,0.2,0.2],
                        vertical_spacing=0.04, subplot_titles=(f'{company} price', 'Acquistion','Disposition'))
        
        
    fig_price.update_layout(height=1000)
    fig_price.add_trace(go.Candlestick(x=data.index,
            open=data['open'],
            high=data['high'],
            low=data['low'],
            close=data['close'],
            name='price'), row=1, col=1)



    for col in data.columns:
        if type(col)!=str:
            if col[0]=='A':
                fig_price.add_trace(go.Bar(x=data.index, y=data[col], name=f'{col[1]}-{col[0]}'), row=2,col=1)
            if col[0]=='D':
                fig_price.add_trace(go.Bar(x=data.index, y=data[col], name=f'{col[1]}-{col[0]}'), row=3,col=1)

    fig_price.update_layout(title=title, xaxis_rangeslider_visible=False, barmode='stack',
    newshape_line_color='black')
    if resample=='D':

        fig_price.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])
    
    fig_price.update_xaxes(spikemode='across+marker')
    if log_scale==True:
        fig_price.update_yaxes(type='log', row=2,col=1)
        fig_price.update_yaxes(type='log',row=3,col=1)
        fig_price.layout.annotations[1].update(text='Acquistion log scale')
        fig_price.layout.annotations[2].update(text='Disposition log scale')


    fig_price.update_traces(xaxis="x1")
    fig_price.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                   ]})
        

In [281]:
plot_towner_on_price_chart(df_towner)

### Only sale and purchase 

In [282]:
def merge_data_sp(trades_data,price_data,resample):
    
    
    df_aord_pivot=pd.DataFrame()
    price_df=price_data.copy()
    trades_df=trades_data.copy()
    trades_df=trades_df.loc[(trades_df['transactionType'] =='S-Sale') | (trades_df['transactionType'] =='P-Purchase')]
    company=trades_df['symbol'].iloc[1]
    
    
    trades_df.reset_index(inplace=True)
    trades_df['filingDate']=pd.DatetimeIndex(trades_df['filingDate']).date
    df_a_or_d=trades_df.groupby(['filingDate','acquistionOrDisposition', 'transactionType'])['securitiesTransacted'].sum()
    df_a_or_d=df_a_or_d.reset_index()
    df_aord_pivot=df_a_or_d.pivot_table(index=['filingDate'], columns=['acquistionOrDisposition','transactionType'],
                                       values='securitiesTransacted').fillna(0)

    df_aord_pivot.index=pd.to_datetime(df_aord_pivot.index)


    if resample!='D':
        df_aord_pivot=df_aord_pivot.resample(resample).agg('sum')
        price_df=price_df.resample(resample).agg({'open':'first','high':'max','low':'min','close':'last', 'volume':'sum'})

    price_towner_df=pd.concat([price_df[['open','high','low','close','volume']],df_aord_pivot], axis=1).fillna(0)
    price_towner_df.attrs['symbol'] = company
    price_towner_df.attrs['freq'] = resample
    
    return price_towner_df

In [283]:
df_sp=merge_data_sp(trades_data,price_data, resample='W')

In [284]:
df_sp

,open,high,low,close,volume,"(A, P-Purchase)","(D, S-Sale)"
2019-01-06,99.550,102.51,97.20,101.93,121969000,0.0,0.0
2019-01-13,101.640,104.88,100.98,102.80,157833100,0.0,0.0
2019-01-20,101.900,107.90,101.26,107.71,155699200,0.0,0.0
2019-01-27,106.750,107.88,104.86,107.17,112636000,0.0,0.0
2019-02-03,106.260,106.48,102.17,102.78,201611200,0.0,0.0
...,...,...,...,...,...,...,...
2024-01-28,400.020,407.01,393.59,403.93,111234300,0.0,0.0
2024-02-04,406.060,415.32,397.21,411.22,164761600,0.0,0.0
2024-02-11,409.900,420.82,402.91,420.55,109311001,0.0,48303.0
2024-02-18,420.555,420.74,403.39,404.06,113544267,0.0,0.0


In [285]:
def plot_sp_on_price_chart(data, log_scale=False):
    
    
    company=data.attrs['symbol']
    resample=data.attrs['freq']
    title=f"Type of insider trades |{resample}| related to {company} price, from {data.index[0]} to {data.index[-1]}"
    fig_price=make_subplots(rows=3,cols=1, row_heights=[0.6,0.2,0.2],
                        vertical_spacing=0.04, subplot_titles=(f'{company} price', 'Acquistion','Disposition'))
        
        
    fig_price.update_layout(height=1000)
    fig_price.add_trace(go.Candlestick(x=data.index,
            open=data['open'],
            high=data['high'],
            low=data['low'],
            close=data['close'],
            name='price'), row=1, col=1)



    for col in data.columns:
        if type(col)!=str:
            if col[0]=='A':
                fig_price.add_trace(go.Bar(x=data.index, y=data[col], name=col[1]), row=2,col=1)
            if col[0]=='D':
                fig_price.add_trace(go.Bar(x=data.index, y=data[col], name=col[1]), row=3,col=1)

    fig_price.update_layout(title=title, xaxis_rangeslider_visible=False, barmode='stack',
    newshape_line_color='black')
    if resample=='D':

        fig_price.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])])
    
    fig_price.update_xaxes(spikemode='across+marker')
    if log_scale==True:
        fig_price.update_yaxes(type='log', row=2,col=1)
        fig_price.update_yaxes(type='log',row=3,col=1)
        fig_price.layout.annotations[1].update(text='Acquistion log scale')
        fig_price.layout.annotations[2].update(text='Disposition log scale')


    fig_price.update_traces(xaxis="x1")
    fig_price.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                   ]})
        

In [286]:
plot_sp_on_price_chart(df_sp, log_scale=True)